# Building a Basic Brick

In this tutorial, we'll go through a simple example of how to build a basic AND brick.  We'll start with some basic imports.  We almost always use NetworkX and Numpy.  The fugu imports are really just for testing our new brick.

In [1]:
import numpy as np
import fugu
from fugu import Scaffold, Brick
from fugu.bricks import Vector_Input
from fugu.backends import snn_Backend
from fugu.scaffold import ChannelSpec, PortSpec, ChannelData, PortData, PortUtil

All bricks should inherit from the `Brick` class.  Bricks that are listed as input bricks should instead inherit from `InputBrick`, which is beyond the scope of this tutorial.  The construction of most brick types is similar; constructing a brick that takes input coding "current" (see below) is a bit different and is beyond the scope of this tutorial.

The `Brick` class provides the framework for the a scaffold to build a neural graph. Subclasses of `Brick` should provide the actual code that will generate the nodes and edges on a graph.  The graph construction should take place within the `build` method.  Let's look at the definition of the parent class `Brick`.

```python 
class Brick(ABC):
    def __init__(self):
        self.name = "Empty Brick"
        self.supported_codings = []

    @abstractmethod
    def build2(self, graph, inputs: dict[str, PortData] = {}):
        pass

```

The first line `class Brick(ABC)` defines the abstract class of `Brick`.  Brick objects inherit from `ABC` which just means that `Brick` is an abstract class that cannot be instantiated on its own; only subclasses may be instantiated.

The ``__init__`` method contains standard instantiation code.  All bricks are expected to have a member property `self.name` that is unique to the brick.  The uniqueness needs to be determined by the scaffold, not by the brick.

The property `self.is_built` is a boolean that is True if the brick has been built (added to the graph).

The property `self.supported_codings` is a list of input codings (strings) that the brick supports.  Since you have the full use of python when you are defining your brick, you can support multiple coding types completely transparent to the user.  A full list of coding types can be found at `fugu.input_coding_types`.

The method `build2` will be called by the scaffold when the graph is to be built.  Arguments are:
- graph: The neuron graph object that we are building onto.
- inputs: A *dict* of input ports. Each *PortData* object describes the structure of one port, along with lists of concrete neuron IDs.

Each brick is responsible for throwing the appropriate errors/warnings if the inputs are not compatible with the brick.

Below is a basic AND brick.

In [2]:
class AND(Brick):
    """
    Brick for performing a logical AND.
    Operation is performed entry-wise, matching based on index.  All codings are supported.
    """

    def __init__(self, name="AND"):
        """
        Args:
            name (str): Name of the brick.  If not specified, a default will be used.  Name should be unique.
        """
        super(AND, self).__init__(name)

        # The brick hasn't been built yet.
        self.is_built = False

        # We just store the name passed at construction.
        self.name = name

    # This method describes the input ports that any AND brick expects.
    # This includes the actual data values, as well as a signal indicating when the
    # source of input is ready for us to procss the data. Since we execute in a single
    # cycle, this signal is simply passed on to the next brick (see build2() later).
    @classmethod
    def input_ports(cls) -> dict[str, PortSpec]:
        port = PortSpec(name='input', minimum=2, maximum=2)  # Must be exactly two ports, no more, no less.
        port.channels['data']     = ChannelSpec(name='data')
        port.channels['complete'] = ChannelSpec(name='complete')
        return {port.name: port}

    # This method describes the output port that any AND brick provides.
    # This includes the data and the 'complete' signal that we pass on.
    @classmethod
    def output_ports(cls) -> dict[str, PortSpec]:
        port = PortSpec(name='output')
        port.channels['data']     = ChannelSpec(name='data', coding=['Raster'])
        port.channels['complete'] = ChannelSpec(name='complete')
        return {port.name: port}

    # This method does the actual network construction. It reads the input ports, wires up
    # some neurons, and provides their identities via the output port.
    def build2(self, graph, inputs: dict[str, PortData] = {}):
        """
        Build AND brick.
        Raises:
            ValueError: If != 2 inputs.  Only 2 inputs are supported.  Error if unsupported mode.
        """
        # Expect two inputs
        if len(inputs) != 2:
            raise ValueError('Only two inputs supported.')

        # Set up convenience variables for accessing our working ports.
        input1, input2 = PortUtil.get_autoports(inputs, 'input', 2)  # Unpack the input ports.
        result = PortUtil.make_ports_from_specs(AND.output_ports())  # Create our output port(s).
        output = result['output']  # Unpack the only actual output port.
        data = output.channels['data']  # Unpack the data channel where our main result goes.

        # Keep the same coding as input 0 for the output
        # This is an arbitrary decision at this point.
        # Generally, your brick will impart some coding, but that isn't the case here.
        data.spec.coding = input1.channels['data'].spec.coding

        # Hook up the signals.
        # We just forward the incoming signal with one cycle of delay.
        complete_node_name = self.generate_neuron_name('complete')
        output.channels['complete'].neurons = [complete_node_name]
        graph.add_node(complete_node_name,
                       index=-1,
                       threshold=0.0,
                       decay=0.0,
                       p=1.0,
                       potential=0.0)
        graph.add_edge(input1.channels['complete'].neurons[0],
                       complete_node_name,
                       weight=1.0,
                       delay=1)

        # Build the computational graph.
        # This is the heart of the brick.
        # The plan is to iterate over our two input data channels in parallel, connecting
        # each pair of inputs via a new neuron that does the AND operation.
        data1 = input1.channels['data'].neurons
        data2 = input2.channels['data'].neurons
        for i in range(min(len(data1), len(data2))):
            operand1 = data1[i]
            operand2 = data2[i]
            # Generate a name for the new neuron, which also happens to be an output neuron.
            and_node_name = self.generate_neuron_name(f"{operand1}_{operand2}")
            data.neurons.append(and_node_name)
            # Create the neuron.
            graph.add_node(and_node_name,
                           index=0,
                           threshold=1.0,
                           decay=1.0,
                           p=1.0,
                           potential=0.0)
            # Make a synapse to the first input neuron.
            graph.add_edge(operand1,
                           and_node_name,
                           weight=0.75,
                           delay=1.0)
            # Make a synapse to the second input neuron.
            graph.add_edge(operand2,
                           and_node_name,
                           weight=0.75,
                           delay=1.0)

        self.is_built = True
        return result

Brick classes provide a means for _reflection_. We can ask them for their port specification as a data structure by calling input_ports() and output_ports(). Or we can ask for a human-readable description.

In [3]:
AND.show_ports()
print()
Vector_Input.show_ports()

Inputs:
  'input'
    index   = 0
    minimum = 2
    maximum = 2
    channels:
      'data'
        Required
      'complete'
        Required
Outputs:
  'output'
    index   = 0
    channels:
      'data'
        coding = ['Raster']
      'complete'

No inputs
Outputs:
  'output'
    index   = 0
    channels:
      'data'
      'begin'
      'complete'


Knowing the ports, we can assemble a scaffold.

In [4]:
scaffold = Scaffold()
I1 = scaffold.add_brick(Vector_Input(np.array([1,0,1,0]), coding='Raster', name='input1'))
I2 = scaffold.add_brick(Vector_Input(np.array([1,1,0,0]), coding='Raster', name='input2'))
A = scaffold.add_brick(AND(), output=True)
scaffold.connect(I1, A)  # Binds I1's default output port to A's first default input port.
scaffold.connect(I2, A)  # Binds I2's default output port to A's second default input port.
scaffold.lay_bricks()
scaffold.summary(verbose=1)

Scaffold is built: True
-------------------------------------------------------
List of Bricks:


Brick No.: 0
Brick Tag: input1-0
Brick Name: input1
{'tag': 'input1-0', 'name': 'input1', 'brick': <fugu.bricks.input_bricks.Vector_Input object at 0x1607c3620>, 'layer': 'input', 'ports': {'output': PortData(spec=PortSpec(name='output', description='', index=0, minimum=1, maximum=1, channels={'data': ChannelSpec(name='data', description='', coding='Raster', shape=(4,), required=True), 'begin': ChannelSpec(name='begin', description='', coding=[], shape=None, required=True), 'complete': ChannelSpec(name='complete', description='', coding=[], shape=None, required=True)}), channels={'data': ChannelData(spec=ChannelSpec(name='data', description='', coding='Raster', shape=(4,), required=True), neurons=['input1-0:(0,)', 'input1-0:(1,)', 'input1-0:(2,)', 'input1-0:(3,)']), 'begin': ChannelData(spec=ChannelSpec(name='begin', description='', coding=[], shape=None, required=True), neurons=['input1-0

Finally, we can simulate the network and view its output.

In [5]:
backend = snn_Backend()
backend_args = {}
backend_args['record'] = 'all'
backend.compile(scaffold, backend_args)
result = backend.run(10)
print(result)

   time  neuron_number
0   0.0            0.0
1   0.0            1.0
2   0.0            6.0
3   0.0            7.0
4   1.0           12.0
